In [95]:
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
import pickle

In [96]:
df = pd.read_csv('../bayes/datos2.csv', delimiter = ";").astype(str)
df.head(3)

,category,label,title,company,profile
0,Administración y gestión de empresas,1,se busca analista de reclutamiento,chazki,procesos de reclutamiento y adquisición de tal...
1,Administración y gestión de empresas,1,se busca analista de reclutamiento,chazki,procesos de reclutamiento y adquisición de tal...
2,Administración y gestión de empresas,1,auxiliar de recursos humanos,zen consultores,esta vacante viene de la bolsa de empleo talen...


In [97]:
col = ['category', 'profile']
df = df[col]
df = df[pd.notnull(df['profile'])]

df.columns = ['category', 'profile']

df['category_id'] = df['category'].factorize()[0]

category_id_df = df[['category', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'category']].values)
df.head(3)

,category,profile,category_id
0,Administración y gestión de empresas,procesos de reclutamiento y adquisición de tal...,0
1,Administración y gestión de empresas,procesos de reclutamiento y adquisición de tal...,0
2,Administración y gestión de empresas,esta vacante viene de la bolsa de empleo talen...,0


In [3]:
etiquetas = [
    'Administración',   'Contabilidad',    'Derecho',
    'Docencia',         'Comunicación',    'Ingeniería',
    'Medicina',         'Enfermería',      'Psicología',
    'Cien_Compu',       'Ayudante'
]
centroides = [
    3, 3, 5,    
    6, 6, 4,
    5, 5, 5,
    3, 4
]

print ('etiquetas', len(etiquetas), 'centrides', len(centroides))

etiquetas 11 centrides 11


In [99]:
i = 0
label = 'Modelos_Entrenados/KMeans_'+str(i)+'_'+etiquetas[0]+'.plk'
Vector= 'Modelos_Entrenados/Vector_'+str(i)+'_'+etiquetas[0]+'.plk'
print(label,'\n',Vector)

Modelos_Entrenados/KMeans_0_Administración.plk 
 Modelos_Entrenados/Vector_0_Administración.plk


In [100]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(stopwords.words('spanish'))

for i in range(0,len(etiquetas)):
    data = df[df.category_id==i]
    
    vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words=my_stop_words)
    features = vectorizer.fit_transform(data.profile).toarray()
    
    model = KMeans(n_clusters=centroides[i], max_iter=600,random_state=0, n_init=25, init='k-means++')
    model.fit(features)
    
    name_vector= 'Modelos_Entrenados/Vector_'+str(i)+'_'+etiquetas[i]+'.plk'
    name_model = 'Modelos_Entrenados/KMeans_'+str(i)+'_'+etiquetas[i]+'.plk'

    pickle.dump(vectorizer, open(name_vector,'wb'))
    pickle.dump(model, open(name_model,'wb'))

---
# Cargar modelos

---

In [1]:
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
import pickle
etiquetas = [
    'Administración',   'Contabilidad',    'Derecho',
    'Docencia',         'Comunicación',    'Ingeniería',
    'Medicina',         'Enfermería',      'Psicología',
    'Cien_Compu',       'Ayudante'
]
centroides = [
    3, 3, 5,    
    6, 6, 4,
    5, 5, 5,
    3, 4
]

In [17]:
for i in range(0,len(etiquetas)):
#for i in range(0,4):
    name_vector= 'Modelos_Entrenados_sub_categorias/Vector_'+str(i)+'_'+etiquetas[i]+'.plk'
    name_model = 'Modelos_Entrenados_sub_categorias/KMeans_'+str(i)+'_'+etiquetas[i]+'.plk'

    print(name_model)

    vectorizer = pickle.load(open(name_vector,'rb'))
    model = pickle.load(open(name_model,'rb'))
    
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()
    
    for x in range(0,centroides[i]):
        print(" \\\\ \\ [%d] &" % x, end=' ')
        for ind in order_centroids[x, :100]:
            print(' %s &' % terms[ind],end=' '),
        print('\n')
    
    print('\n\n\n')

Modelos_Entrenados_sub_categorias/KMeans_0_Administración.plk
 \\ \ [0] &  aira &  proceso selección &  cargo &  través aira &  postular &  selección &  descripción &  oferta &  etapa &  mail &  proceso &  través &  selección proceso &  descripción empresa &  preguntas &  funciones cargo &  misión &  realiza &  condiciones oferta &  misión cargo &  solo &  ingresar aira &  realiza través &  preguntas pruebas &  selección realiza &  aira contestar &  contestar preguntas &  descripción proceso &  beneficios &  contestar &  postular solo &  plataforma reclutamiento &  perfil ajusta &  experiencia postulación &  aira plataforma &  diseñado mejorar &  necesitas postular &  ajusta buscando &  postulación postular &  reclutamiento diseñado &  si vemos &  ajusta &  buscando contactaremos &  vemos perfil &  mejorar experiencia &  diseñado &  vemos &  solo necesitas &  presencial &  ingresar &  aira seguir &  condiciones &  plataforma &  postulación &  revisar &  contactaremos &  seguir etapa & 

In [111]:
for i in range(0,len(etiquetas)):
    name_vector= 'Modelos_Entrenados/Vector_'+str(i)+'_'+etiquetas[i]+'.plk'
    name_model = 'Modelos_Entrenados/KMeans_'+str(i)+'_'+etiquetas[i]+'.plk'

    print(name_vector)
    print(name_model)

Modelos_Entrenados/Vector_0_Administración.plk
Modelos_Entrenados/KMeans_0_Administración.plk
Modelos_Entrenados/Vector_1_Contabilidad.plk
Modelos_Entrenados/KMeans_1_Contabilidad.plk
Modelos_Entrenados/Vector_2_Derecho.plk
Modelos_Entrenados/KMeans_2_Derecho.plk
Modelos_Entrenados/Vector_3_Docencia.plk
Modelos_Entrenados/KMeans_3_Docencia.plk
Modelos_Entrenados/Vector_4_Comunicación.plk
Modelos_Entrenados/KMeans_4_Comunicación.plk
Modelos_Entrenados/Vector_5_Ingeniería.plk
Modelos_Entrenados/KMeans_5_Ingeniería.plk
Modelos_Entrenados/Vector_6_Medicina.plk
Modelos_Entrenados/KMeans_6_Medicina.plk
Modelos_Entrenados/Vector_7_Enfermería.plk
Modelos_Entrenados/KMeans_7_Enfermería.plk
Modelos_Entrenados/Vector_8_Psicología.plk
Modelos_Entrenados/KMeans_8_Psicología.plk
Modelos_Entrenados/Vector_9_Cien_Compu.plk
Modelos_Entrenados/KMeans_9_Cien_Compu.plk
Modelos_Entrenados/Vector_10_Ayudante.plk
Modelos_Entrenados/KMeans_10_Ayudante.plk


In [112]:
vector_Admin = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_0_Administración.plk','rb'))
model__Admin = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_0_Administración.plk','rb'))
vector_Conta = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_1_Contabilidad.plk','rb'))
model__Conta = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_1_Contabilidad.plk','rb'))
vector_Derec = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_2_Derecho.plk','rb'))
model__Derec = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_2_Derecho.plk','rb'))
vector_Docen = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_3_Docencia.plk','rb'))
model__Docen = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_3_Docencia.plk','rb'))
vector_Comun = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_4_Comunicación.plk','rb'))
model__Comun = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_4_Comunicación.plk','rb'))
vector_Ingen = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_5_Ingeniería.plk','rb'))
model__Ingen = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_5_Ingeniería.plk','rb'))
vector_Medic = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_6_Medicina.plk','rb'))
model__Medic = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_6_Medicina.plk','rb'))
vector_Enfer = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_7_Enfermería.plk','rb'))
model__Enfer = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_7_Enfermería.plk','rb'))
vector_Psico = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_8_Psicología.plk','rb'))
model__Psico = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_8_Psicología.plk','rb'))
vector_Ci_Co = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_9_Cien_Compu.plk','rb'))
model__Ci_Co = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_9_Cien_Compu.plk','rb'))
vector_Ayudn = pickle.load(open('Modelos_Entrenados_sub_categorias/Vector_10_Ayudante.plk','rb'))
model__Ayudn = pickle.load(open('Modelos_Entrenados_sub_categorias/KMeans_10_Ayudante.plk','rb'))